In [348]:
from pyzotero import zotero as pyzt
import pandas as pd
import sqlite3
import yaml
import re
import requests
import json

In [160]:
# Get your zotero configuration
with open('zotero.yaml', 'r') as file:
    cf = yaml.safe_load(file)

In [4]:
# Access SAEF group library
zot = pyzt.Zotero(library_id = cf['library']['id'], library_type=cf['library']['type'], api_key=cf['library']['key'])

In [22]:
# Import Filemaker Person keys 
pids = pd.read_csv('../../data/Peoplekey.csv', usecols=[0, 1]) 
pids.columns = ['NameKey', 'ID_Person']
pids['NameKey'] = pids['NameKey'].str.replace(' ', '')

In [5]:
# Retrieve all top-level items
saef_library = zot.everything(zot.top())

In [151]:
# Build a dataframe of outputs with a particular tag
for x in saef_library:
    found = [ x.get('tag') == 'to ngo' for x in x['data']['tags'] ]
    if any(found):
        print(f"{x['data']['tags']} - {x['data']['key']}")
        


[{'tag': 'to ngo'}] - PBMB2GRL
[{'tag': 'to ngo'}] - ELGX2CFV
[{'tag': 'to ngo'}] - JEJTLVNY
[{'tag': 'to ngo'}] - JVHK6JV7
[{'tag': 'to ngo'}] - 53FDL8FV


In [61]:
# https://pyzotero.readthedocs.io/en/latest/#zotero.Zotero.collections
def matched_library(library):
    items = library
    result = pd.DataFrame(columns=['key', 'itemType', 'title', 'rights', 'pubyr', 'name', 'id_person'])
                          
    for x in items:
        item  = x['data']
        pubyr = re.findall( '\d{4}', item['date'] )[0]  # Match 4 consequtive numbers in a string i.e. YYYY
        extra = item['extra']
        saef_author_list = re.findall('saef:.*', extra) # Use saef keyword as an identifier
        
        for names in saef_author_list:
            names_tidy  = re.sub('saef:', '', names, flags=re.IGNORECASE)
            names_split = names_tidy.strip().split(';') # Create a list of saef names
            
            for name in names_split:
                # Find FileMaker person key
                id_person = pids[pids.NameKey == name]['ID_Person'].to_string()
                # 'key', ['Item Type', 'Title', 'Journal', 'publication', 'rights', 'date', author, id]
                store = [item['key'], item['itemType'], item['title'], item['rights'], pubyr, name, id_person]
                result.loc[len(result)] = store
    return result

In [385]:
'Multiple'.str.contains('SCAR', regex=False)

AttributeError: 'str' object has no attribute 'str'

In [65]:
# Get a Filemaker people_Outputs ready object
bucket = matched_library(saef_library)

In [66]:
bucket[bucket.id_person == 'Series([], )']

,key,itemType,title,rights,pubyr,name,id_person
2,9B6LFADN,radioBroadcast,Antarctic ecosystems under double pressure,oa,2023,AleksTerauds,"Series([], )"
27,GRSJHYBS,journalArticle,Indigenous and introduced Collembola differ in...,OA,2023,RebeccaHallas,"Series([], )"
28,GRSJHYBS,journalArticle,Indigenous and introduced Collembola differ in...,OA,2023,LauraPhillips,"Series([], )"
43,SBZSJ2BS,webpage,Biggest bacterium ever discovered shakes our v...,oa,2022,ChrisGreening,"Series([], )"
44,ZY2CE7DI,webpage,Largest bacterium ever discovered has an unexp...,oa,2022,ChrisGreening,"Series([], )"
72,7DLNC5KP,presentation,Atmospheric carbon monoxide oxidation: from en...,,2022,ChrisGreening,"Series([], )"
97,BKA5P6QQ,newspaperArticle,Electricity from thin air: an enzyme from bact...,,2023,ChrisGreening,"Series([], )"
104,7BT9PL7W,conferencePaper,Patterns of humans’ preferences for choosing a...,,2022,ZhaoXiang,"Series([], )"
159,GQMVINZZ,conferencePaper,Structure and Dynamics of Antarctic Governance,,2022,Larissa LubianaBotelho,"Series([], )"
160,S7TTC5RU,conferencePaper,Wet deposition in shallow convection over the ...,,2022,TeharahAlinajadtabrizi,"Series([], )"


In [38]:
bucket.tail(n=30)

,key,itemType,title,rights,pubyr,name,id_person
192,6YMJTGGL,newspaperArticle,"As Antarctica changes, so must our strategies ...",,2022,JasmineLee,61 E6069ABD-ACFB-A84A-9848-FF144AFE175A
193,NHYZMZ9N,webpage,We studied how the Antarctic ice sheet advance...,oa,2022,RichardJones,136 43F2CB88-19F4-2146-902C-36DB913DB8CE
194,SBWH4BF3,webpage,New research shows how the Antarctic ice sheet...,oa,2022,RichardJones,136 43F2CB88-19F4-2146-902C-36DB913DB8CE
195,6U3GW3VG,webpage,We studied how the Antarctic ice sheet advance...,,2022,RichardJones,136 43F2CB88-19F4-2146-902C-36DB913DB8CE
196,PQP3B78A,newspaperArticle,'You get to impact the world': Researcher usin...,oa,2022,KateHelmstedt,80 81E17FF3-F45C-074F-9A24-83BC9189D10E
197,8A7D3K8W,webpage,Antarctica: coming to a postcode near you : Ec...,oa,2022,StevenChown,148 88080486-B8DB-7D4E-BAA1-1AC0A85139A4
198,EN9Z56GU,webpage,Antarctica: coming to postcode near you,oa,2022,StevenChown,148 88080486-B8DB-7D4E-BAA1-1AC0A85139A4
199,MSGXBVWJ,webpage,Report highlights impact of changes in Antarctica,oa,2022,StevenChown,148 88080486-B8DB-7D4E-BAA1-1AC0A85139A4
200,FP64P9S7,webpage,Emperor penguins face extinction in Antarctica...,oa,2022,StevenChown,148 88080486-B8DB-7D4E-BAA1-1AC0A85139A4
201,SQWV2LKG,journalArticle,Introduced and invasive alien species of Anta...,oa,2023,RachelLeihy,132 DFD26A28-0C1F-6A4E-8917-5941D16C1AF2


In [68]:
pids[pids['NameKey']=='ChristopherGreening']

,NameKey,ID_Person
27,ChristopherGreening,5A831488-9ECA-7F46-8322-FD2FCDCE1D3B
